# Model Validation (ESL)
This notebook expands on the concepts presented in chapeter 7 *Model Assessment and Selection* of the *Elements of Statistical Learning* book. The topics are not necessarily examined in order of appearence on the chapter, rather following a convenient thread of thought.

## Extra-sample and In-sample errors
It is interesting to elaborate the concept of *Extra-sample* and *In-sample* error covered in sections 7.4 and followinng of the chapter. Using the same notation of the books, let
$$ \mathcal{T} = \{(x_1, y_1, \ldots, x_N, y_N)\} $$
be the training set
and considere a re-sampling of the points (here a make a little change in the notation my $y_i'$ corresponds to the $Y^0_i$ of the book).
$$ \mathcal{T}' = \{ (x_1, y_i'), \ldots, (x_N, y_N') \} $$
For a given error function $L(y_i, \hat{f})$ we can calculate the *training error*
$$ \bar{err} = \sum_{i=1}^{N}{L(y_i, \hat{f}(x_i))} $$
the *in-sample error*
$$ Err_{in} = \sum_{i=1}^{N}{E_{\mathbf{y}'}[L(y_i', \hat{f}(x_i))]} $$
and the *optimism*
$$ Err_{in} - \bar{err} $$


In [51]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

def linear_model(beta, X, noise=0):
    '''Computes and returns beta.T*X + noise'''
    return (np.matmul(X,beta) + noise*np.random.randn(np.shape(X)[0])).reshape(-1,1)

n_samples = 50
n_features = 1
noise = 30
X = np.random.randn(n_samples, n_features)*5
beta  = np.random.randn(n_features)*40
y = linear_model(beta,X,noise)

model = LinearRegression()
model.fit(X,y)
tilde_y = model.predict(X)

n_resamples = 30
Y_res = np.zeros([n_samples, n_resamples])
errs_in_sample = np.zeros([n_resamples,1])
# resample
for i in range(n_resamples):
    ys = linear_model(beta,X,noise)
    Y_res[:,i] = ys.reshape(n_samples)
    errs_in_sample[i] = mean_squared_error(ys, model.predict(X))

in_sample = np.mean(errs_in_sample)
train_err = mean_squared_error(y, model.predict(X))
print("Training error {0}".format(train_err))
print("In sample mean error {0}".format(in_sample))
print("Optimism {0}".format(in_sample-train_err))

Training error 949.2792719709629
In sample mean error 962.510783443715
Optimism 13.231511472752118


We can have from the above analysis a partial confirmation that the optimism is, on average, positive and thus that the training error is somehow too good (*i.e.*, too optimistic) as predictor of the actual error. This is quite intuitive, but I don't see as obvious the analytical argument for it.

It seems to me that the same phenomenon that makes training error biased downward, should be in place when we use training error as predictor of the test error. In this case the phenomenon should be even higher because we are not trying to estimate the same $x_i$ used for constructing the prediction, but we are trying to predict the unknown function in points $x'$ that (presumibly) we have not yet seen.

The claim made in the text is that
$$ E_{\mathbf{y}}[Err_{in}-\bar{err}] = \frac{2}{N}\sum_{i=1}^{N}{Cov(\hat{y}_i, y_i)}$$
where th expectation is with respect to the training set (more specifically its $\mathbf{y}$ part).
The proof of this results (outlined in Exercise 7.4) could shed some lights on the nature of this situation and it is therefore useful to pursue to better understand the whole phenomenon. (**TODO Exercise 7.4**). 

Leaving, for the moment, aside the proof of above result, we can write the following relation (pretty general)
$$ E_{\mathbf{y}}[Err_{in}] = E_{\mathbf{y}}[\bar{err}] + \frac{2}{N}\sum_{i=1}^{N}{Cov(\hat{y}_i, y_i)}$$
Recalling the $Err_{in}$ is defined by the expectation over re-sampling on the same points $x_i$, it is no surprise that its expectation (with respect to the training set) does not depend from the actual re-sampling. What is $E[Err_{in}]$ actually measuring then?

### IID additive noise
When
$$ y_i = f(x_i) + \eta $$
with $\eta \sim \mathcal{N}(0,\sigma)$ a normal gaussian noise with zero mean and standard deviation $\sigma$, the sum of covariances simplifies to $d\sigma$ (**TODO prove it**) where $d$ is the number of components of $x_i$, that is $x_i\in\mathbb{R}^d$. Hence
$$ E_{\mathbf{y}}[Err_{in}] = E_{\mathbf{y}}[\bar{err}] + \frac{2}{N}d\sigma^2$$
Let see if experimentally this relation is confirmed (notice that to have a better measure we should perform several times the above experiment changing the training set)

In [53]:
eff_dof = (noise**2)*2*n_features / n_samples
Cp_stat = train_err + eff_dof
print("E[Err_in] ~= {0}".format(Cp_stat))
print("Estimate diff: {0}".format(in_sample-(train_err+eff_dof)))

E[Err_in] ~= 985.2792719709629
Estimate diff: -22.768488527247882


The whole idea of *in sample error* analysis is to be able to use it to predict the prediction error (*i.e.*, the error on the, unknown, distribution over $\mathcal{X}$). More precisely, since we have the training error, a good estimate of the optimism can be used to evaluate the prediction error (from above formula one sees that this approach estimate the in sample error, here the assumption is that this is a good estimate of the prediction error). It makes thus sense to find good estimators $\hat{Err_{in}}$ of the in-sample error.

## $C_p$ statistic
The $C_p$ statistic is appropriate when a $d$ parameter model is fit under additive IID noise using least square, the above estimate of the *effective number of parameters is used*
$$ \hat{Err_{in}} = \bar{err} + 2\frac{d}{N}\hat{\sigma}^2 $$
where $\hat{\sigma}^2$ is an estimate of the variance of the noise $\eta$.

## Akaike Information Criterion (AIC)
Let start from the general form of the criterion where the model selection processo maximizes the quantity
$$ AIC(\alpha) = \bar{err}(\alpha) + 2\frac{d(\alpha)}{N}\hat{\sigma}^2 $$
where $\alpha$ is a parameter indexing a family of models $f_\alpha(x)$. As the model changes in the family the training error $err(\alpha)$ changes (with more complex model likely to give smaller errors) and also the number of parameters $d(\alpha)$ (which measures the complexity of the model). In this case we see that each new parameter (and increase by $1$ of $d(\alpha)$) is worth only if it makes the error increase by more than $2\hat{\sigma}^2/N$, this says (informally) that each new parameter must "buy" at least two variances in terms of the objective function (otherwise the increase in $d$ is not worth).